In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

# Load the dataset
df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Datasets/FinalActivityDataset_Fahrenheit.csv')

# Encode categorical variables
label_encoders = {}
categorical_columns = ['Time_of_Day', 'AQI_Category', 'Suggested_Activity']
for col in categorical_columns:
    le = LabelEncoder()
    df[col] = le.fit_transform(df[col])
    label_encoders[col] = le

# Feature Engineering: Add a new feature for outdoor-friendly conditions
df['Outdoor_Friendly'] = ((df['AQI'] < 50) & (df['Time_of_Day'].between(8, 18))).astype(int)

# Define features and target variable
X = df[['Age', 'Time_of_Day', 'AQI', 'Temperature', 'Precipitation', 'Outdoor_Friendly']]
y = df['Suggested_Activity']

# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Hyperparameter tuning for RandomForestClassifier
param_grid = {
    'n_estimators': [50, 100, 200],
    'max_depth': [None, 10, 20],
    'min_samples_split': [2, 5, 10]
}

grid_search = GridSearchCV(RandomForestClassifier(random_state=42), param_grid, cv=3)
grid_search.fit(X_train, y_train)

# Best model
best_model = grid_search.best_estimator_

# Evaluate the model
y_pred = best_model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print(f'Model Accuracy: {accuracy:.2f}')

# Function to recommend activity based on user input
def recommend_activity(age, time_of_day, aqi, temperature, precipitation):
    # Encode the input time_of_day
    time_of_day_encoded = label_encoders['Time_of_Day'].transform([time_of_day])[0]

    # Calculate outdoor-friendly condition
    outdoor_friendly = int((aqi < 50) and (6 <= int(time_of_day) <= 18))

    # Create a DataFrame with the input features
    input_data = pd.DataFrame({
        'Age': [age],
        'Time_of_Day': [time_of_day_encoded],
        'AQI': [aqi],
        'Temperature': [temperature],
        'Precipitation': [precipitation],
        'Outdoor_Friendly': [outdoor_friendly]
    })

    # Predict the activity
    predicted_activity_encoded = best_model.predict(input_data)[0]

    # Decode the predicted activity
    predicted_activity = label_encoders['Suggested_Activity'].inverse_transform([predicted_activity_encoded])[0]

    return predicted_activity

# Test cases to evaluate the recommend_activity function
test_cases = [
    # Warm and Sunny Day (Outdoor Activity Expected)
    {"age": 25, "time_of_day": "14", "aqi": 30, "temperature": 28, "precipitation": 0.1},

    # Cold and Rainy Day (Indoor Activity Expected)
    {"age": 40, "time_of_day": "10", "aqi": 60, "temperature": 10, "precipitation": 4.5},

    # Evening with Clear Sky (Stargazing Expected)
    {"age": 35, "time_of_day": "20", "aqi": 20, "temperature": 15, "precipitation": 0.0},

    # Hot and Humid Day (Outdoor Sports Expected)
    {"age": 18, "time_of_day": "16", "aqi": 50, "temperature": 32, "precipitation": 0.2},

    # Cold and Snowy Day (Indoor Activity Expected)
    {"age": 50, "time_of_day": "12", "aqi": 70, "temperature": 2, "precipitation": 3.0},

    # Moderate Weather (Picnic Expected)
    {"age": 30, "time_of_day": "13", "aqi": 40, "temperature": 22, "precipitation": 0.0},

    # Early Morning with Low AQI (Jogging Expected)
    {"age": 28, "time_of_day": "8", "aqi": 10, "temperature": 18, "precipitation": 0.0},

    # Late Night with High AQI (Indoor Activity Expected)
    {"age": 45, "time_of_day": "22", "aqi": 90, "temperature": 12, "precipitation": 0.0},

    # Hot and Dry Day (Outdoor Sports Expected)
    {"age": 22, "time_of_day": "15", "aqi": 55, "temperature": 35, "precipitation": 0.0},

    # Cool and Breezy Evening (Walk Expected)
    {"age": 60, "time_of_day": "18", "aqi": 25, "temperature": 20, "precipitation": 0.0},

    # Rainy Afternoon (Indoor Activity Expected)
    {"age": 33, "time_of_day": "14", "aqi": 65, "temperature": 18, "precipitation": 2.5},

    # Clear Morning with Moderate AQI (Picnic Expected)
    {"age": 27, "time_of_day": "9", "aqi": 45, "temperature": 24, "precipitation": 0.0},

    # Cold Evening with Low AQI (Stargazing Expected)
    {"age": 38, "time_of_day": "19", "aqi": 15, "temperature": 8, "precipitation": 0.0},

    # Hot Afternoon with High AQI (Indoor Activity Expected)
    {"age": 20, "time_of_day": "14", "aqi": 85, "temperature": 34, "precipitation": 0.0},

    # Mild Weather with Light Rain (Walk Expected)
    {"age": 55, "time_of_day": "17", "aqi": 35, "temperature": 19, "precipitation": 1.0},

    # Freezing Morning with Snow (Indoor Activity Expected)
    {"age": 42, "time_of_day": "7", "aqi": 75, "temperature": -2, "precipitation": 4.0},

    # Warm Evening with Low AQI (Picnic Expected)
    {"age": 29, "time_of_day": "18", "aqi": 30, "temperature": 26, "precipitation": 0.0},

    # Hot and Windy Day (Outdoor Sports Expected)
    {"age": 19, "time_of_day": "13", "aqi": 40, "temperature": 33, "precipitation": 0.0},

    # Cool Night with High AQI (Indoor Activity Expected)
    {"age": 48, "time_of_day": "21", "aqi": 95, "temperature": 10, "precipitation": 0.0},

    # Mild Afternoon with Light Precipitation (Walk Expected)
    {"age": 37, "time_of_day": "15", "aqi": 50, "temperature": 21, "precipitation": 0.5},

    # Extreme AQI (300) - Hazardous Air Quality (Indoor Activity Expected)
    {"age": 30, "time_of_day": "12", "aqi": 300, "temperature": 25, "precipitation": 0.0},

    # Extreme AQI (500) - Hazardous Air Quality (Indoor Activity Expected)
    {"age": 40, "time_of_day": "16", "aqi": 500, "temperature": 28, "precipitation": 0.0},

    # High AQI (200) - Unhealthy Air Quality (Indoor Activity Expected)
    {"age": 35, "time_of_day": "10", "aqi": 200, "temperature": 20, "precipitation": 0.0},

    # High AQI (150) - Unhealthy for Sensitive Groups (Indoor Activity Expected)
    {"age": 50, "time_of_day": "14", "aqi": 150, "temperature": 22, "precipitation": 0.0},

    # High AQI (100) - Moderate Air Quality (Outdoor Activity Expected)
    {"age": 25, "time_of_day": "18", "aqi": 100, "temperature": 24, "precipitation": 0.0}
]

# Run the test cases
for i, test_case in enumerate(test_cases, 1):
    age = test_case["age"]
    time_of_day = test_case["time_of_day"]
    aqi = test_case["aqi"]
    temperature = test_case["temperature"]
    precipitation = test_case["precipitation"]

    recommended_activity = recommend_activity(age, time_of_day, aqi, temperature, precipitation)
    print(f"Test Case {i}: Age={age}, Time_of_Day={time_of_day}, AQI={aqi}, Temperature={temperature}, Precipitation={precipitation}")
    print(f"Recommended Activity: {recommended_activity}\n")

Model Accuracy: 1.00
Test Case 1: Age=25, Time_of_Day=14, AQI=30, Temperature=28, Precipitation=0.1
Recommended Activity: Have a picnic

Test Case 2: Age=40, Time_of_Day=10, AQI=60, Temperature=10, Precipitation=4.5
Recommended Activity: Go for a jog

Test Case 3: Age=35, Time_of_Day=20, AQI=20, Temperature=15, Precipitation=0.0
Recommended Activity: Stargazing

Test Case 4: Age=18, Time_of_Day=16, AQI=50, Temperature=32, Precipitation=0.2
Recommended Activity: Stay indoors

Test Case 5: Age=50, Time_of_Day=12, AQI=70, Temperature=2, Precipitation=3.0
Recommended Activity: Stay indoors

Test Case 6: Age=30, Time_of_Day=13, AQI=40, Temperature=22, Precipitation=0.0
Recommended Activity: Have a picnic

Test Case 7: Age=28, Time_of_Day=8, AQI=10, Temperature=18, Precipitation=0.0
Recommended Activity: Go for a jog

Test Case 8: Age=45, Time_of_Day=22, AQI=90, Temperature=12, Precipitation=0.0
Recommended Activity: Stay indoors

Test Case 9: Age=22, Time_of_Day=15, AQI=55, Temperature=35, 